In [57]:
try:
    __IPYTHON__
    USING_IPYTHON = True
    %load_ext autoreload
    %autoreload 2
except NameError:
    USING_IPYTHON = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Argparse

In [26]:
import argparse
ap = argparse.ArgumentParser()
ap.add_argument('project_root', help='')
ap.add_argument('--mrp-data-dir', default='data', help='')
ap.add_argument('--graphviz-sub-dir', default='visualization/graphviz', help='')
ap.add_argument('--train-sub-dir', default='training', help='')
ap.add_argument('--companion-sub-dir', default='companion')

ap.add_argument('--mrp-file-extension', default='.mrp')
ap.add_argument('--companion-file-extension', default='.conllu')
ap.add_argument('--graphviz-file-template', default='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/{}/{}.mrp/{}.png')
arg_string = """
    /data/proj29_ds1/home/slai/mrp2019
"""
arguments = [arg for arg_line in arg_string.split(r'\\n') for arg in arg_line.split()]

In [27]:
if USING_IPYTHON:
    args = ap.parse_args(arguments)
else:
    args = ap.parse_args()

In [28]:
args

Namespace(companion_file_extension='.conllu', companion_sub_dir='companion', graphviz_file_template='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/{}/{}.mrp/{}.png', graphviz_sub_dir='visualization/graphviz', mrp_data_dir='data', mrp_file_extension='.mrp', project_root='/data/proj29_ds1/home/slai/mrp2019', train_sub_dir='training')

#### Library imports

In [29]:
import json
import logging
import os
import pprint
import string

from collections import Counter
from collections import defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import plot_util

#### ipython notebook specific imports

In [30]:
if USING_IPYTHON:
    # matplotlib config
    %matplotlib inline

In [31]:
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler()])
logger = logging.getLogger(__name__)  # pylint: disable=invalid-name
logger.setLevel(logging.INFO)

### Constants

In [32]:
UNKWOWN = 'UNKWOWN'

### Load data

In [36]:
train_dir = os.path.join(args.project_root, args.mrp_data_dir, args.train_sub_dir)
frameworks = [sub_dir for sub_dir in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, sub_dir))]
frameworks

['ucca', 'psd', 'eds', 'dm', 'amr']

In [37]:
framework2dataset2mrp_jsons = {}
for framework in tqdm(frameworks, desc='frameworks'):
    dataset2mrp_jsons = {}
    framework_dir = os.path.join(train_dir, framework)
    dataset_names = os.listdir(framework_dir)
    
    for dataset_name in tqdm(dataset_names, desc='dataset_name'):
        mrp_jsons = []
        if not dataset_name.endswith(args.mrp_file_extension):
            continue
        with open(os.path.join(framework_dir, dataset_name)) as rf:
            for line in rf:
                mrp_json = json.loads(line.strip())
                if framework == 'ucca' and 'nodes' in mrp_json and 'input' in mrp_json:
                    input_text = mrp_json['input']
                    nodes = mrp_json['nodes']
                    for i, node in enumerate(nodes):
                        if 'anchors' not in node:
                            continue
                        text_segments = []
                        for anchor in node['anchors']:
                            text_segments.append(input_text[anchor.get('from', -1): anchor.get('to', -1)])
                        mrp_json['nodes'][i]['label'] = ''.join(text_segments)
                        
                mrp_jsons.append(mrp_json)
        dataset_name = dataset_name.split('.')[0]
        dataset2mrp_jsons[dataset_name] = mrp_jsons
                
    framework2dataset2mrp_jsons[framework] = dataset2mrp_jsons

frameworks: 100%|██████████| 5/5 [00:18<00:00,  3.18s/it]t/s]


In [38]:
for framework in framework2dataset2mrp_jsons:
    logger.info(framework)
    logger.info(list(framework2dataset2mrp_jsons[framework].keys()))

INFO:__main__:ucca
INFO:__main__:['wiki', 'ewt']
INFO:__main__:psd
INFO:__main__:['wsj']
INFO:__main__:eds
INFO:__main__:['wsj']
INFO:__main__:dm
INFO:__main__:['wsj']
INFO:__main__:amr
INFO:__main__:['xinhua', 'wsj', 'wiki', 'wb', 'rte', 'proxy', 'mt09sdl', 'lorelei', 'fables', 'dfb', 'dfa', 'cctv', 'bolt', 'amr-guidelines']


### Data Preprocessing companion

In [50]:
dataset2cid2parse = {}
companion_dir = os.path.join(args.project_root, args.mrp_data_dir, args.companion_sub_dir)
for framework in os.listdir(companion_dir):
    framework_dir = os.path.join(companion_dir, framework)
    if not os.path.isdir(framework_dir):
        continue
    
    logger.info('framework {} found'.format(framework))
    for dataset in tqdm(os.listdir(framework_dir), desc='dataset'):
        if not dataset.endswith(args.companion_file_extension):
            continue
        dataset_name = dataset.split('.')[0].rstrip(string.digits)
        cid2parse = {}
        with open(os.path.join(framework_dir, dataset)) as rf:
            parse = []
            for line in rf:
                line = line.strip()
                if not line:
                    cid2parse[cid] = parse
                    parse = []
                    cid = ''
                elif line.startswith('#'):
                    cid = line[1:]
                else:
                    parse.append(line.split('\t'))
        dataset2cid2parse[dataset_name] = cid2parse

INFO:__main__:framework amr found
dataset: 100%|██████████| 13/13 [00:03<00:00,  3.26it/s]
INFO:__main__:framework dm found
dataset: 100%|██████████| 5/5 [00:01<00:00,  3.80it/s]
INFO:__main__:framework ucca found
dataset: 100%|██████████| 6/6 [00:00<00:00, 24.27it/s]


In [51]:
dataset2cid2parse.keys()

dict_keys(['amr-guidelines', 'bolt', 'cctv', 'dfa', 'dfb', 'fables', 'lorelei', 'mt09sdl', 'proxy', 'rte', 'wb', 'wiki', 'xinhua', 'wsj', 'ewt'])

In [53]:
# Some data is missing
'20003001' in dataset2cid2parse['wsj']

False

### Plot companion data

In [55]:
dataset = 'wiki'
dataset = 'wsj'
framework = 'dm'
framework = 'eds'
framework = 'psd'
framework = 'ucca'
framework = 'amr'
mrp_index = 26

parse = None
while not parse:
    mrp_index += 1
    cid = framework2dataset2mrp_jsons[framework][dataset][mrp_index]['id']
    parse = dataset2cid2parse[dataset].get(cid)
    
# print sentence
print(mrp_index, ' '.join([word_record[1] for word_record in parse]))

# print corresponding mrp png link
for framework in frameworks:
    if dataset not in framework2dataset2mrp_jsons[framework]:
        continue
    if len(framework2dataset2mrp_jsons[framework][dataset]) <= mrp_index:
        continue
    if framework2dataset2mrp_jsons[framework][dataset][mrp_index]['id'] == cid:
        print(args.graphviz_file_template.format(framework, dataset, cid))

# 
def plot_parse(parse, args, dataset, cid):
    """plot parse and print png filename"""
    parse_dg = plot_util.parse_to_directed_graph(parse)
    dataset_dir = os.path.join(args.graphviz_sub_dir, dataset)
    image_path = os.path.join(dataset_dir, cid + '.png')
    os.makedirs(dataset_dir, exist_ok=True)
    plot_util.directed_graph_to_graphviz_image(parse_dg, image_path)
    return os.path.join('http://localhost:8000/files/proj29_ds1/home/slai/mrp2019', image_path)
    
parse_plot_filename = plot_parse(parse, args, dataset, cid)
print(parse_plot_filename)

28 “ But you have to recognize that these events took place 35 years ago .
http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/psd/wsj.mrp/20003029.png
http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/eds/wsj.mrp/20003029.png
http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/dm/wsj.mrp/20003029.png
http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/amr/wsj.mrp/20003029.png
http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/wsj/20003029.png


### Build model

In [76]:
from typing import Dict, Iterator, List

import numpy as np

import torch
import torch.optim as optim
from allennlp.common.file_utils import cached_path
from allennlp.data import Instance
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.fields import SequenceLabelField, TextField
from allennlp.data.iterators import BucketIterator
from allennlp.data.token_indexers import SingleIdTokenIndexer, TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.seq2seq_encoders import (PytorchSeq2SeqWrapper,
                                               Seq2SeqEncoder)
from allennlp.modules.text_field_embedders import (BasicTextFieldEmbedder,
                                                   TextFieldEmbedder)
from allennlp.modules.token_embedders import Embedding
from allennlp.nn.util import (get_text_field_mask,
                              sequence_cross_entropy_with_logits)
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.training.trainer import Trainer

In [ ]:
torch.manual_seed(1)

In [78]:
from model.allennlp_crf_tutorial import (LstmTagger, PosDatasetReader)

In [79]:
reader = PosDatasetReader()
train_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/training.txt'))
validation_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/validation.txt'))
vocab = Vocabulary.from_instances(train_dataset + validation_dataset)

INFO:allennlp.common.file_utils:https://raw.githubusercontent.com/allenai/allennlp/master/tutorials/tagger/training.txt not found in cache, downloading to /tmp/tmpxuqfddt8
93B [00:00, 52948.32B/s]             
INFO:allennlp.common.file_utils:copying /tmp/tmpxuqfddt8 to cache at /home/slai/.allennlp/cache/c3e1f451545a79cf7582dec24d072db6f5bb0d1ae24a924d03c9944516e16b60.47b1193282cbd926a1b602cc6d5a22324cfab24e669ca04f1ff4851a35c73393
INFO:allennlp.common.file_utils:creating metadata file for /home/slai/.allennlp/cache/c3e1f451545a79cf7582dec24d072db6f5bb0d1ae24a924d03c9944516e16b60.47b1193282cbd926a1b602cc6d5a22324cfab24e669ca04f1ff4851a35c73393
INFO:allennlp.common.file_utils:removing temp file /tmp/tmpxuqfddt8
2it [00:00, 2932.05it/s]
INFO:allennlp.common.file_utils:https://raw.githubusercontent.com/allenai/allennlp/master/tutorials/tagger/validation.txt not found in cache, downloading to /tmp/tmpuz51_aem
93B [00:00, 52155.40B/s]             
INFO:allennlp.common.file_utils:copying /tm

In [86]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)

In [88]:
if torch.cuda.is_available():
    cuda_device = 1
    model = model.cuda(cuda_device)
else:
    cuda_device = -1

In [89]:
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("sentence", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=1000,
                  cuda_device=cuda_device)

In [90]:
trainer.train()

INFO:allennlp.training.trainer:Beginning training.
INFO:allennlp.training.trainer:Epoch 0/999
INFO:allennlp.training.trainer:Peak CPU memory usage MB: 7287.308
INFO:allennlp.training.trainer:GPU 0 memory usage MB: 10877
INFO:allennlp.training.trainer:GPU 1 memory usage MB: 11886
INFO:allennlp.training.trainer:GPU 2 memory usage MB: 1076
INFO:allennlp.training.trainer:GPU 3 memory usage MB: 10
INFO:allennlp.training.trainer:GPU 4 memory usage MB: 11886
INFO:allennlp.training.trainer:GPU 5 memory usage MB: 10
INFO:allennlp.training.trainer:GPU 6 memory usage MB: 11
INFO:allennlp.training.trainer:GPU 7 memory usage MB: 11835
INFO:allennlp.training.trainer:Training
accuracy: 0.3333, loss: 1.1685 ||: 100%|██████████| 1/1 [00:00<00:00, 18.56it/s]
INFO:allennlp.training.trainer:Validating
accuracy: 0.3333, loss: 1.1592 ||: 100%|██████████| 1/1 [00:00<00:00, 53.11it/s]
INFO:allennlp.training.tensorboard_writer:                    Training |  Validation
INFO:allennlp.training.tensorboard_writer

INFO:allennlp.training.trainer:GPU 4 memory usage MB: 11886
INFO:allennlp.training.trainer:GPU 5 memory usage MB: 10
INFO:allennlp.training.trainer:GPU 6 memory usage MB: 11
INFO:allennlp.training.trainer:GPU 7 memory usage MB: 11835
INFO:allennlp.training.trainer:Training
accuracy: 0.3333, loss: 1.1391 ||: 100%|██████████| 1/1 [00:00<00:00, 105.77it/s]
INFO:allennlp.training.trainer:Validating
accuracy: 0.3333, loss: 1.1316 ||: 100%|██████████| 1/1 [00:00<00:00, 299.02it/s]
INFO:allennlp.training.tensorboard_writer:                    Training |  Validation
INFO:allennlp.training.tensorboard_writer:gpu_2_memory_MB |  1078.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_0_memory_MB |  10877.000  |       N/A
INFO:allennlp.training.tensorboard_writer:loss            |     1.139  |     1.132
INFO:allennlp.training.tensorboard_writer:gpu_1_memory_MB |  11886.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_3_memory_MB |    10.000  |       N/A
INFO:allennlp.trai

accuracy: 0.3333, loss: 1.1167 ||: 100%|██████████| 1/1 [00:00<00:00, 123.81it/s]
INFO:allennlp.training.trainer:Validating
accuracy: 0.3333, loss: 1.1107 ||: 100%|██████████| 1/1 [00:00<00:00, 305.91it/s]
INFO:allennlp.training.tensorboard_writer:                    Training |  Validation
INFO:allennlp.training.tensorboard_writer:gpu_2_memory_MB |  1078.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_0_memory_MB |  10877.000  |       N/A
INFO:allennlp.training.tensorboard_writer:loss            |     1.117  |     1.111
INFO:allennlp.training.tensorboard_writer:gpu_1_memory_MB |  11886.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_3_memory_MB |    10.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_4_memory_MB |  11886.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_7_memory_MB |  11835.000  |       N/A
INFO:allennlp.training.tensorboard_writer:cpu_memory_MB   |  7298.468  |       N/A
INFO:allennlp.training.tensorboard_writer

INFO:allennlp.training.tensorboard_writer:gpu_0_memory_MB |  10877.000  |       N/A
INFO:allennlp.training.tensorboard_writer:loss            |     1.100  |     1.095
INFO:allennlp.training.tensorboard_writer:gpu_1_memory_MB |  11886.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_3_memory_MB |    10.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_4_memory_MB |  11886.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_7_memory_MB |  11835.000  |       N/A
INFO:allennlp.training.tensorboard_writer:cpu_memory_MB   |  7298.472  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_5_memory_MB |    10.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_6_memory_MB |    11.000  |       N/A
INFO:allennlp.training.tensorboard_writer:accuracy        |     0.333  |     0.333
INFO:allennlp.training.trainer:Epoch duration: 0:00:00.262895
INFO:allennlp.training.trainer:Estimated training time remaining: 0:04:24
INFO:allennlp.training.trainer

INFO:allennlp.training.tensorboard_writer:gpu_7_memory_MB |  11835.000  |       N/A
INFO:allennlp.training.tensorboard_writer:cpu_memory_MB   |  7298.472  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_5_memory_MB |    10.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_6_memory_MB |    11.000  |       N/A
INFO:allennlp.training.tensorboard_writer:accuracy        |     0.333  |     0.333
INFO:allennlp.training.trainer:Epoch duration: 0:00:00.247024
INFO:allennlp.training.trainer:Estimated training time remaining: 0:04:18
INFO:allennlp.training.trainer:Epoch 17/999
INFO:allennlp.training.trainer:Peak CPU memory usage MB: 7298.472
INFO:allennlp.training.trainer:GPU 0 memory usage MB: 10877
INFO:allennlp.training.trainer:GPU 1 memory usage MB: 11886
INFO:allennlp.training.trainer:GPU 2 memory usage MB: 1078
INFO:allennlp.training.trainer:GPU 3 memory usage MB: 10
INFO:allennlp.training.trainer:GPU 4 memory usage MB: 11886
INFO:allennlp.training.trainer:GPU 5 memor

...

INFO:allennlp.training.trainer:Peak CPU memory usage MB: 7299.076
INFO:allennlp.training.trainer:GPU 0 memory usage MB: 10877
INFO:allennlp.training.trainer:GPU 1 memory usage MB: 11886
INFO:allennlp.training.trainer:GPU 2 memory usage MB: 1078
INFO:allennlp.training.trainer:GPU 3 memory usage MB: 10
INFO:allennlp.training.trainer:GPU 4 memory usage MB: 11886
INFO:allennlp.training.trainer:GPU 5 memory usage MB: 10
INFO:allennlp.training.trainer:GPU 6 memory usage MB: 11
INFO:allennlp.training.trainer:GPU 7 memory usage MB: 11835
INFO:allennlp.training.trainer:Training
accuracy: 1.0000, loss: 0.0186 ||: 100%|██████████| 1/1 [00:00<00:00, 104.69it/s]
INFO:allennlp.training.trainer:Validating
accuracy: 1.0000, loss: 0.0185 ||: 100%|██████████| 1/1 [00:00<00:00, 444.03it/s]
INFO:allennlp.training.tensorboard_writer:                    Training |  Validation
INFO:allennlp.training.tensorboard_writer:gpu_2_memory_MB |  1078.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_0_me

INFO:allennlp.training.trainer:GPU 5 memory usage MB: 10
INFO:allennlp.training.trainer:GPU 6 memory usage MB: 11
INFO:allennlp.training.trainer:GPU 7 memory usage MB: 11835
INFO:allennlp.training.trainer:Training
accuracy: 1.0000, loss: 0.0184 ||: 100%|██████████| 1/1 [00:00<00:00, 113.37it/s]
INFO:allennlp.training.trainer:Validating
accuracy: 1.0000, loss: 0.0184 ||: 100%|██████████| 1/1 [00:00<00:00, 405.33it/s]
INFO:allennlp.training.tensorboard_writer:                    Training |  Validation
INFO:allennlp.training.tensorboard_writer:gpu_2_memory_MB |  1078.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_0_memory_MB |  10877.000  |       N/A
INFO:allennlp.training.tensorboard_writer:loss            |     0.018  |     0.018
INFO:allennlp.training.tensorboard_writer:gpu_1_memory_MB |  11886.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_3_memory_MB |    10.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_4_memory_MB |  11886.000  |     

accuracy: 1.0000, loss: 0.0182 ||: 100%|██████████| 1/1 [00:00<00:00, 403.45it/s]
INFO:allennlp.training.tensorboard_writer:                    Training |  Validation
INFO:allennlp.training.tensorboard_writer:gpu_2_memory_MB |  1078.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_0_memory_MB |  10877.000  |       N/A
INFO:allennlp.training.tensorboard_writer:loss            |     0.018  |     0.018
INFO:allennlp.training.tensorboard_writer:gpu_1_memory_MB |  11886.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_3_memory_MB |    10.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_4_memory_MB |  11886.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_7_memory_MB |  11835.000  |       N/A
INFO:allennlp.training.tensorboard_writer:cpu_memory_MB   |  7299.076  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_5_memory_MB |    10.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_6_memory_MB |    11.000  |       N/A

INFO:allennlp.training.tensorboard_writer:gpu_1_memory_MB |  11886.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_3_memory_MB |    10.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_4_memory_MB |  11886.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_7_memory_MB |  11835.000  |       N/A
INFO:allennlp.training.tensorboard_writer:cpu_memory_MB   |  7299.076  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_5_memory_MB |    10.000  |       N/A
INFO:allennlp.training.tensorboard_writer:gpu_6_memory_MB |    11.000  |       N/A
INFO:allennlp.training.tensorboard_writer:accuracy        |     1.000  |     1.000
INFO:allennlp.training.trainer:Epoch duration: 0:00:00.261968
INFO:allennlp.training.trainer:Estimated training time remaining: 0:00:00
INFO:allennlp.training.trainer:Epoch 999/999
INFO:allennlp.training.trainer:Peak CPU memory usage MB: 7299.076
INFO:allennlp.training.trainer:GPU 0 memory usage MB: 10877
INFO:allennlp.training.tra

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/slai/.pyenv/versions/mrp/lib/python3.7/site-packages/en_core_web_sm -->
/home/slai/.pyenv/versions/mrp/lib/python3.7/site-packages/spacy/data/en_core_web_sm
You can now load the model via spacy.load('en_core_web_sm')
['DET', 'NN', 'V', 'DET', 'NN']


In [91]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)
tag_logits = predictor.predict("The dog ate the apple")['tag_logits']
tag_ids = np.argmax(tag_logits, axis=-1)
print([model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])

['DET', 'NN', 'V', 'DET', 'NN']
